# Matrix Factorization

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/recommendation/collaborative_filtering(linear)/matrix_factorization.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

In [3]:
def matrix_factorization(R, K, steps=5000, alpha=0.0002, beta=0.02):
    N = len(R)
    M = len(R[0])
    P = np.random.rand(N, K)
    Q = np.random.rand(M, K)

    Q = Q.T
    for step in range(steps):
        for i in range(N):
            for j in range(M):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])
                    for k in range(K):
                        P[i][k] += alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] += alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = np.dot(P, Q)
        e = 0
        for i in range(N):
            for j in range(M):
                if R[i][j] > 0:
                    e += pow(R[i][j] - np.dot(P[i, :], Q[:, j]), 2)
                    for k in range(K):
                        e += (beta / 2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        if e < 0.001:
            break
    return P, Q.T

# 例：ユーザー-アイテム評価行列
R = [
     [5, 3, 0, 1],
     [4, 0, 0, 1],
     [1, 1, 0, 5],
     [1, 0, 0, 4],
     [0, 1, 5, 4],
    ]

N = len(R)
M = len(R[0])
K = 2

nP, nQ = matrix_factorization(R, K)
nR = np.dot(nP, nQ.T)
print(nR)

[[5.05561559 2.76135037 5.58241341 0.99797621]
 [3.92189666 2.14342579 4.44934968 1.00052311]
 [1.12327952 0.64091874 3.73028812 4.96613689]
 [0.93633491 0.53332243 3.02493783 3.97857502]
 [2.72886133 1.51060787 4.84215693 4.02357172]]
